In [29]:
#!/usr/bin/python 

VERSION = 1.00

#---------------------------------
# BAGEL:  Bayesian Analysis of Gene EssentaLity
# (c) Traver Hart, 02/2015.
# modified 9/2015
# modeified by Asad on 3/2017
# Free to modify and redistribute with attribution
#---------------------------------

import numpy as np
import scipy.stats as stats
import sys, getopt

helptext = ('\n'
		   'BAGEL.py -i [fold change file] -o [output file] -e [reference essentials] -n [reference nonessentials] -c [columns to test]\n' 
		   '\n'
		   '  from the Bayesian Analysis of Gene EssentiaLity (BAGEL) suite\n'
		   '  Version ' + str(VERSION) + '\n' 
		   '\n'
		   '  required options:\n' 
		   '     -i  [fold change file]         Tab-delmited file of reagents and fold changes.  See documentation for format.\n' 
		   '     -o  [output file]              Output filename\n' 
		   '     -e  [reference essentials]     File with list of training set of essential genes\n' 
		   '     -n  [reference nonessentials]  File with list of training set of nonessential genes\n' 
		   '     -c  [columns to test]          comma-delimited list of columns in input file to include in analyisis\n' 
		   '\n' 
		   '  other options:\n'
		   '     --numiter=N                    Number of bootstrap iterations (default 1000)\n'
		   '     -h, --help                     Show this help text\n'
		   '\n'
		   '  Example:\n' 
		   '  BAGEL.py -i foldchange_file -o experiment.bf -e essentials_training_set -n nonessentials_training_set -c 1,2,3\n'
		   '\n'
		   '  Calculates a log2 Bayes Factor for each gene; positive BFs indicate confidence that the gene is essential.\n'
		   '  writes to [output file]: gene name, mean Bayes Factor across all iterations, std deviation of BFs, and number of iterations\n'
		   '  in which the gene was part of the test set (and a BF was calculated[output file]\n' 
		   '\n')

NUM_BOOTSTRAPS = 1000

try:
	opts, args = getopt.getopt(sys.argv[1:], "hi:o:c:e:n:", ["numiter=","help"])
except getopt.GetoptError:
	print(helptext)
	sys.exit(2)
for opt, arg in opts:
	if opt in ( '-h', '--help'):
		print(helptext)
		sys.exit()
	elif opt == '-i':
		foldchangefile = arg
	elif opt == '-o':
		outfilename = arg
	elif opt == '-e':
		ess_ref = arg
	elif opt == '-n':
		non_ref = arg
	elif opt == '-c':
		columns = arg.split(',')
	elif opt == '--numiter':
		NUM_BOOTSTRAPS = int(arg)




BAGEL.py -i [fold change file] -o [output file] -e [reference essentials] -n [reference nonessentials] -c [columns to test]

  from the Bayesian Analysis of Gene EssentiaLity (BAGEL) suite
  Version 0.91

  required options:
     -i  [fold change file]         Tab-delmited file of reagents and fold changes.  See documentation for format.
     -o  [output file]              Output filename
     -e  [reference essentials]     File with list of training set of essential genes
     -n  [reference nonessentials]  File with list of training set of nonessential genes
     -c  [columns to test]          comma-delimited list of columns in input file to include in analyisis

  other options:
     --numiter=N                    Number of bootstrap iterations (default 1000)
     -h, --help                     Show this help text

  Example:
  BAGEL.py -i foldchange_file -o experiment.bf -e essentials_training_set -n nonessentials_training_set -c 1,2,3

  Calculates a log2 Bayes Factor for each ge

SystemExit: 2

/Users/arahman/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [30]:
foldchangefile='test/BAGEL_FC.txt'
outfilename = 'test/out.txt'
ess_ref = 'training_essentials.txt' 
non_ref = 'training_nonessential.txt'
NUM_BOOTSTRAPS = 1000
columns=[1,2]


In [44]:


column_list = [int(c) for c in columns]

FC_THRESH = 2**-7
genes={}
fc = {}

def round_to_hundredth(x):
	return np.round( x*100) / 100.0

def bootstrap_resample(X, n=None):
    """ Bootstrap resample an array_like
    Parameters
    ----------
    X : array_like
      data to resample
    n : int, optional
      length of resampled array, equal to len(X) if n==None
    Results
    -------
    returns X_resamples

    adapted from
    Dated 7 Oct 2013
    http://nbviewer.ipython.org/gist/aflaxman/6871948
    """
    if n == None:
            n = len(X)

    resample_i = floor(random.rand(n)*len(X)).astype(int)
    X_resample = X[resample_i]
    return X_resample


#
# LOAD FOLDCHANGES
#


fin  = open(foldchangefile)
skipfields = fin.readline().rstrip().split('\t')
for i in column_list:
    print("Using column:  " + skipfields[i+1])
for line in fin:
    fields = line.rstrip().split('\t')
    gsym = fields[1]
    genes[ gsym ]=1
    if ( not gsym in fc ):
        fc[gsym]=[]    # initialize dict entry as a list
    for i in column_list:
        fc[gsym].append( float(fields[i + 1]))		# per user docs, GENE is column 0, first data column is col 1.
genes_array = np.asarray( list(genes.keys()))
gene_idx = arange(len( genes ))
#print("Number of gRNA loaded:  " + str( len(genes_array) ))
#print("genes_array:  " + str(genes_array))
# print("Genes:  " + str(genes))
# print("Bayes:	" + str(fc))
# print("Gene Arrays: " + str(genes_array))
# print("Number of unique genes:  " + str( len(genes) ))

#
# DEFINE REFERENCE SETS
#
coreEss = []
fin = open(ess_ref)
for line in fin:
	coreEss.append( line.rstrip().split('\t')[0] )
fin.close()
coreEss=array(coreEss)
print("Number of reference essentials: " + str(len(coreEss)))

nonEss = []
fin = open(non_ref)
for line in fin:
    nonEss.append( line.rstrip().split('\t')[0] )
fin.close()
nonEss = array(nonEss)
print("Number of reference nonessentials: " + str(len(nonEss)))

#
# INITIALIZE BFS
#
bf = {}
# print(genes_array)
for g in genes_array:
    bf[g]=[]

#
# BOOTSTRAP ITERATIONS
#
print("Iter, TrainEss, TrainNon, TestSet")
sys.stdout.flush()
for loop in range(NUM_BOOTSTRAPS):
    #
    # bootstrap resample from gene list to get the training set
    #
    gene_train_idx = bootstrap_resample(gene_idx)
    #
    # test set for this iteration is everything not selected in bootstrap resampled training set
    #
    gene_test_idx = setxor1d(gene_idx, gene_train_idx)
    #
    # define essential and nonessential training sets:  arrays of indexes
    #
    
#     print (gene_train_idx)
#     print (coreEss)
    train_ess = where( in1d( genes_array[gene_train_idx], coreEss))[0]
    train_non = where( in1d( genes_array[gene_train_idx], nonEss))[0]
    print("{},{},{},{}".format(str(loop), len(train_ess),len(train_non), len(gene_test_idx)))
    sys.stdout.flush()
    #
    # define ess_train: vector of observed fold changes of essential genes in training set
    #
    ess_train_fc_list_of_lists = [ fc[x] for x in genes_array[gene_train_idx[train_ess]] ]
#     print(len(ess_train_fc_list_of_lists))
    ess_train_fc_flat_list = [obs for sublist in ess_train_fc_list_of_lists for obs in sublist]
#     print(len(ess_train_fc_list_of_lists))
    #
    # define non_train vector of observed fold changes of nonessential genes in training set
    #
    non_train_fc_list_of_lists = [ fc[x] for x in genes_array[gene_train_idx[train_non]] ]
#     print(len(non_train_fc_list_of_lists))
    non_train_fc_flat_list = [obs for sublist in non_train_fc_list_of_lists for obs in sublist]
#     print(len(non_train_fc_flat_list))
    
    if len(ess_train_fc_flat_list) > 0 and len(non_train_fc_flat_list) > 0:
        #
        # calculate empirical fold change distributions for both
        #
        kess = stats.gaussian_kde( ess_train_fc_flat_list )
        knon = stats.gaussian_kde( non_train_fc_flat_list )
        #
        # define empirical upper and lower bounds within which to calculate BF = f(fold change)
        #
        x = arange(-10,2,0.01)
        nonfitx = knon.evaluate(x)
        # define lower bound empirical fold change threshold:  minimum FC where knon is above threshold
        f = where( nonfitx > FC_THRESH)
        xmin = round_to_hundredth( min(x[f]) )
        # define upper bound empirical fold change threshold:  minimum value of log2(ess/non)
        subx = arange( xmin, max(x[f]), 0.01)
        logratio_sample = log2( kess.evaluate(subx) / knon.evaluate(subx) )
        f = where( logratio_sample == logratio_sample.min() )
        xmax = round_to_hundredth( subx[f] )
        #
        # round foldchanges to nearest 0.01
        # precalculate logratios and build lookup table (for speed)
        #
        logratio_lookup = {}
        for i in arange(xmin, xmax+0.01, 0.01):
            logratio_lookup[round(i*100)] = log2( kess.evaluate(i) / knon.evaluate(i) )
        #
        # calculate BFs from lookup table for withheld test set
        #
        for g in genes_array[gene_test_idx]:
            foldchanges = array( fc[g] )
            foldchanges[foldchanges<xmin]=xmin
            foldchanges[foldchanges>xmax]=xmax
            bayes_factor = sum( [ logratio_lookup[ round( x * 100 ) ] for x in foldchanges ] )
            bf[g].append(bayes_factor)


fout = open(outfilename, 'w')
fout.write('GENE\tBF\tSTD\tNumObs\n')
for g in sorted( bf.keys() ):
    num_obs = len( bf[g] )
    bf_mean = mean( bf[g] )
    bf_std  = std( bf[g] )
    bf_norm = ( bf[g] - bf_mean ) / bf_std
    #dstat, pval = stats.kstest( bf_norm, 'norm')
    fout.write('{0:s}\t{1:4.3f}\t{2:4.3f}\t{3:d}\n'.format( g, bf_mean, bf_std, num_obs ) )
fout.close()


Using column:  Replicate-1
Using column:  Replicate-2
Number of reference essentials: 361
Number of reference nonessentials: 928
Iter, TrainEss, TrainNon, TestSet
0,11,4,149
1,15,5,148
2,12,3,142
3,11,4,140
4,9,6,161
5,7,2,150
6,10,5,146
7,14,4,153
8,11,1,145
9,14,1,165
10,9,5,153
11,8,6,144
12,11,1,154
13,9,6,151
14,7,6,137
15,11,0,152
16,8,6,150
17,7,3,142
18,10,2,150
19,3,3,160
20,6,0,154
21,7,6,143
22,11,3,157
23,18,1,143
24,12,3,155
25,10,7,144
26,14,4,158
27,11,4,147
28,14,2,150
29,9,0,146
30,12,3,145
31,10,1,150
32,11,1,150
33,10,7,149
34,18,3,154
35,11,5,140
36,11,6,148
37,8,4,145
38,15,7,149
39,14,6,163
40,9,1,146
41,12,3,138
42,12,11,153
43,11,3,150
44,12,6,153
45,8,3,147
46,6,3,159
47,13,2,148
48,11,1,153
49,7,2,152
50,15,4,137
51,10,1,150
52,7,5,152
53,15,5,160
54,10,4,152
55,7,2,158
56,9,2,151
57,12,1,159
58,7,1,153
59,10,2,151
60,12,2,150
61,17,4,154
62,11,2,147
63,6,7,164
64,12,3,142
65,15,4,152
66,7,3,147
67,6,0,149
68,9,2,153
69,18,4,150
70,18,5,151
71,10,6,155
72,15,5

KeyboardInterrupt: 